# Frame Unwrapping

## Default

In this case there is a 1:1 correspondence between source pulses and neutron pulses propagated to the sample and detectors.

In [ ]:
from frameunwrapping import default_frame_diagram
default_frame_diagram().show()

In the figure above the index `i` labels source pulses.
We define:

- $T_0^i$ is the `event_time_zero` recorded in an `NXevent_data` group.
  These times are indicated by the vertical dotted lines.
- $T_0^{i+1} = T_0^i + L_0$ where the frame length $L_0$ is defined by $L_0 = 1/f_0$, given a source frequency $f_0$.
- $\Delta T_0$ is the offset from $T_0^i$ at which the neutrons are "emitted".
  This may be zero (or half the pulse length) if the full pulse length is used, but choppers such as resolution choppers may extract a section of the pulse that is not aligned with the start of the full pulse.
  This offset can also be used to take into account a potential difference between the timing system's definition of the pulse time and the actual beginning of the neutron pulse exiting, e.g., the moderator.
- The black solid line within the first pulse (blue) indicates a neutron detected at $T_0^{i+1} + \Delta t$.
  $\Delta t$ is the `event_time_offset` in an `NXevent_data` group.
  This value is recorded for every neutron and gives the offset from the latest (previous) `event_time_zero` ($T_0^j$), i.e., the time difference to the previous vertical dotted line.

To compute the time-of-flight for a neutron, we need to identify which source pulse it orginated from.
Consider the shaded vertical band above, indicating the time during which arriving neutrons are associated with $T_0^{i+1}$.
For, e.g., detector 1 when then observe:

- First (small `event_time_offset` $\Delta t$, to the left of the dashed black line) we see the slowest neutrons from N (in this case N=2) source pulses earlier.
- Then (larger `event_time_offset` $\Delta t$, to the right of the dashed black line) we see the slowest neutrons from N-1 (in this case N-1=1) source pulses earlier.
- Typically there is is an intermediate region where no neutrons should be able to traverse the chopper cascade.
  Neutrons detected in this time interval must thus be background from other sources.

To compute the time-of-flight we add an integer multiple of the frame length to `event_time_offset`.
Within a given frame (indicated above by a band between two dotted vertical lines, such as the grey shaded band) there is a *pivot time*:
Neutrons with `event_time_offset` *before* the pivot time originated one source frame *before* neutrons *after* the pivot time.
As illustrated in the figure, the pivot time $t_\text{pivot}$ depends on the detector or rather the distance of the detector from the scattering position.

The pivot time can be computed, e.g., by defining the minimum wavelength $\lambda_{\text{min}}$ that can propagate through the chopper cascade to reach a detector.
This is indicated above by the dashed black line.
The computation of the time-of-flight can then proceed as follows (see also illustration below):

1. Given $\lambda_\text{min}$, we can compute the the corresponding $t_{\text{min}}$ (`tof_min`), i.e., the time the fastest neutrons take from the source to the detector.
   `tof_min` is detector-dependent.
2. The pivot time `time_offset_pivot` is computed as $t_\text{pivot} = (\Delta T_0 + t_{\text{min}})\mod L_0$.
3. The time-of-flight $t_\text{tof}$ is computed as
   $$
   t_{\text{tof}} =  t_{\text{min}} - t_\text{pivot} +\Delta t + 
       \begin{cases}
       L_0, & \text{for } \Delta t \lt t_\text{pivot}\\
       0, & \text{for } \Delta t \ge t_\text{pivot}
       \end{cases}
   $$
   
Note that there are other valid definitions of $t_\text{pivot}$, differing only in how neutrons in the intermediate "background" region (region between two pulses) are mapped to frames.

In [ ]:
from scippneutron.tof import frames
import scipp as sc
sc.show_graph(frames.to_tof(), simplified=True)

## Pulse-skipping

In [ ]:
from frameunwrapping import frame_skipping_diagram
frame_skipping_diagram().show()

## Wavelength-frame multiplication